In [1]:
import requests
from bs4 import BeautifulSoup
import json

In [2]:
file_keys = open('keys.txt', 'r')
keys = [line[:-1] for line in file_keys]
file_keys.close()

In [3]:
col = ['time', 'summary', 'sunriseTime', 'sunsetTime', 'moonPhase', 'precipType', 'temperatureHigh', 'temperatureHighTime', 'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime', 'apparentTemperatureLow', 'apparentTemperatureLowTime', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windBearing', 'windGust', 'windGustTime', 'cloudCover', 'uvIndex', 'uvIndexTime', 'visibility', 'temperatureMin', 'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin', 'apparentTemperatureMinTime', 'apparentTemperatureMax', 'apparentTemperatureMaxTime']

In [4]:
#key = '07a44f54a47e093817bd70285349aac6'
hurl = 'https://api.darksky.net/forecast/'
latitude  = '10.845'
longitude  = '106.645'
start = 883630800 #1998/1/1 5:00:00 AM UTC
end = 1514696400 #2017/12/31 5:00:00 AM UTC
query = '?exclude=currently,hourly,minutely,alerts,flags&units=ca'
secondsPerDate = 24*3600
NUMAPICALLS = 'X-Forecast-API-Calls'

In [5]:
f = open('weather_data.csv', 'w')
for item in col[:-1]:
    f.write(item+',')
f.write(col[-1]+'\n')

err = open('error_data.txt', 'w')

In [6]:
idx = 0

while start <= end:
    right = True
    #request weather data by day
    url = hurl + keys[idx] + '/' + latitude + ',' + longitude + ',' + str(start) + query
    r = requests.get(url)
    try:
        data = json.loads(r.text)
    except Exception as e:
        err.write(str(start)+'\n')
        right = False        
    
    #write data
    if right:
        for key in col[:-1]:            
            f.write(str(data['daily']['data'][0].get(key, '')).replace(',', '|')+',')
        f.write(str(data['daily']['data'][0].get(col[-1], '')).replace(',', '|')+'\n')
    
    #increase day
    start += secondsPerDate
    
    #maximum of request each key per day is 1000
    flag = False
    try:
        flag = int(r.headers[NUMAPICALLS]) >= 1000            
    except Exception as e:
        idx += 1
        start -= secondsPerDate
        
    if flag:
        idx += 1
        #out of key
        if idx >= len(keys):
            break
    
f.close()
err.close()